# Dask Array

Материалы:
* Макрушин С.В. Лекция 11: Dask
* https://docs.dask.org/en/latest/array.html
* JESSE C. DANIEL. Data Science with Python and Dask. 

## Задачи для совместного разбора

In [16]:
import dask.array as da
import h5py
import numpy as np
# import pygraphviz

ModuleNotFoundError: No module named 'pygraphviz'

1. Создайте массив размерностью 1000 на 300000, заполненный числами из стандартного нормального распределения. Исследуйте основные характеристики полученного массива.

In [84]:
%%time
arr_da = da.random.normal(0, 1, size=(10000, 300_000), chunks=(1000, 30_000))
arr_da

Wall time: 10 ms


dask.array<normal, shape=(10000, 300000), dtype=float64, chunksize=(1000, 30000), chunktype=numpy.ndarray>

2. Посчитайте сумму квадратов элементов массива, созданного в задаче 1. Создайте массив `np.array` такого же размера и сравните скорость решения задачи с использование `da.array` и `np.array`

In [85]:
%%time
np.power(arr_da, 2).sum().compute()

Wall time: 1min 11s


2999994618.8570037

In [9]:
%%time
da.power(arr_da, 2).sum().compute()

Wall time: 8.32 s


300033717.4435707

3. Визуализируйте граф вычислений для задачи 2.

In [19]:
da.power(arr_da, 2).sum().visualize()

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

## Лабораторная работа 11

In [21]:
import dask.array as da
import h5py
import numpy as np

1. Считайте датасет `recipe` из файла `minutes_n_ingredients_full.hdf5` в виде `dask.array`. Укажите аргумент `chunks=(100_000, 3)` при создании массива. Выведите на экран основную информацию о массиве.

In [81]:
with h5py.File("11_dask_array_data/minutes_n_ingredients_full.hdf5", "r") as hdf:
	data = da.asarray(hdf["recipe"], chunks=(100_000, 3))

In [82]:
data

dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(100000, 3), chunktype=numpy.ndarray>

2. Вычислите среднее значение по каждому столбцу, кроме первого. 

In [83]:
data.compute()

ValueError: Dset_id is not a dataset id (dset_id is not a dataset ID)

In [67]:
data[:, 1].mean().compute()

ValueError: Dset_id is not a dataset id (dset_id is not a dataset ID)

In [64]:
data[:, 2].mean().compute()

ValueError: Dset_id is not a dataset id (dset_id is not a dataset ID)

3. Исследуйте, как влияет значение аргумента `chunks` при создании `dask.array` на скорость выполнения операции поиска среднего. 

4. Выберите рецепты, время выполнения которых меньше медианного значения

5. Посчитайте количество каждого из возможных значений кол-ва ингредиентов

6. Найдите максимальную продолжительность рецепта. Ограничьте максимальную продолжительность рецептов сверху значением, равному 75% квантилю.

7. Создайте массив `dask.array` из 2 чисел, содержащих ваши предпочтения относительно времени выполнения рецепта и кол-ва ингредиентов. Найдите наиболее похожий (в смысле $L_1$) рецепт из имеющихся в датасете.

8. Работая с исходным файлом в формате `hdf5`, реализуйте алгоритм подсчета среднего значения в блочной форме и вычислите с его помощью среднее значение второго столбца в массиве.

Блочный алгоритм вычислений состоит из двух частей:
1. загрузка фрагмента за фрагментом данных по `blocksize` элементов и проведение вычислений на этим фрагментом;
2. агрегация результатов вычислений на различных фрагментах для получения результата на уровне всего набора данных.

Важно: при работе с `h5py` в память загружаются не все элементы, а только те, которые запрашиваются в данный момент